In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import math
import time

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

import sys
sys.path.append('..')
from detector.input_pipeline import PoseResidualNetworkPipeline

# Build a graph

In [ ]:
tf.reset_default_graph()

files = [
    '/home/dan/datasets/COCO/multiposenet/train/shard-0010.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0020.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0030.tfrecords',
]

with tf.device('/cpu:0'):
    pipeline = PoseResidualNetworkPipeline(files, is_training=True, batch_size=10)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()

# Show an image

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run([features, labels])
    
    heatmaps = output[0]
    binary_masks = output[1]

In [ ]:
i = 1

cmap = plt.cm.get_cmap('autumn')
new_cmap = cmap(np.arange(cmap.N))
new_cmap[:, -1] = np.sqrt(np.linspace(0, 1, cmap.N))  # set alpha
cmap = ListedColormap(new_cmap)  # create new colormap

_, axes = plt.subplots(nrows=17, ncols=2, sharex=True, sharey=True, figsize=(4, 50))

for j in range(17):
    
    axes[j, 0].imshow(heatmaps[i, :, :, j], cmap=cmap)
    axes[j, 1].imshow(binary_masks[i, :, :, j])